In [235]:
import geopandas
import pandas as pd
import matplotlib
import math
import random
from matplotlib import pyplot as plt
import numpy
import geopandas as gpd
from shapely import Point
%matplotlib inline

# avoiding Type 3 fonts in matplotlib plots
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

font = {'size'   : 20}

matplotlib.rc('font', **font)
matplotlib.rc('lines', linewidth=2.0)
matplotlib.rc('lines', markersize=8)

numpy.set_printoptions(threshold=numpy.inf)

pd.set_option('display.max_rows', 500)

In [245]:
# configure simulation
input_file = 'data/gis_incidences_nona.pkl'
number_of_simulations = 5000
a = 2
lk_identifier = (284, 0, 132)
#starting_points = ((51, 42), (135, 142), (55, 250))
#starting_points = ((351, 342), (435, 442), (355, 550))
starting_points = ((351, 342), (435, 442))

# calculate starting points based on geographic coordinates


# create grid
width = 1000 #215
height = 1000 #295
grid_side = 3

gis_incidences_nona=pd.read_pickle(input_file)

#print(gis_incidences_nona.crs)

#gis_incidences_nona = gis_incidences_nona.to_crs('epsg:3035')

#print(gis_incidences_nona.get('geometry')[0].centroid.distance(gis_incidences_nona.get('geometry')[132].centroid) / 1000)

# width: 640km => 215 grid entries
# height: 876km => 295 grid entries
# => approx. 3km x 3km grid

grid = numpy.zeros((height, width), dtype=[('d1', 'int64'), ('d2', 'int64'), ('d3', 'int64'), ('x', 'int64'), ('y', 'int64'), ('z', 'int64')])

In [237]:
# simulate movements
for j in range(0, len(starting_points)):
    i = 0
    while i <= number_of_simulations:
        angle = random.uniform(0, 2 * math.pi)
        x = random.uniform(0, 1)
        d = (math.pow(x, (-1 / (a - 1)))) / 110 # convert distance to km, germany is approx. 876km high and 8 units in the current map
        pos_x = ((starting_points[j][0] * grid_side) + grid_side / 2) + (d * math.cos(angle))
        pos_y = ((starting_points[j][1] * grid_side) + grid_side / 2) + (d * math.sin(angle))

        if (0 <= pos_x < grid_side * width) and (0 <= pos_y < grid_side * height) \
                and (not ((math.floor(pos_x / grid_side) == starting_points[j][0]) and (math.floor(pos_y / grid_side) == starting_points[j][1]))):
            grid[math.floor(pos_y / grid_side)][math.floor(pos_x / grid_side)][j + 3] += 1
            i += 1

# simulate movements
#i = 0
#while i <= number_of_simulations:
#    angle = random.uniform(0, 2 * math.pi)
#    x = random.uniform(0, 1)
#    d = (math.pow(x, (-1 / (a - 1)))) / 110 # convert distance to km, germany is approx. 876km high and 8 units in the current map
#    pos_x = ((starting_point_x * grid_side) + grid_side / 2) + (d * math.cos(angle))
#    pos_y = ((starting_point_y * grid_side) + grid_side / 2) + (d * math.sin(angle))
#
#    while not ((0 <= pos_x <= grid_side * width) and (0 <= pos_y <= grid_side * height)):
#        angle = random.uniform(0, 2 * math.pi)
#        pos_x = ((starting_point_x * grid_side) + grid_side / 2) + (d * math.cos(angle))
#        pos_y = ((starting_point_y * grid_side) + grid_side / 2) + (d * math.sin(angle))
#
#    grid[math.floor(pos_x / grid_side)][math.floor(pos_y / grid_side)][0] += 1
#    i += 1

#print(grid)

In [238]:
# Function to calculate the distance between two points
def calculate_distance(x1, y1, x2, y2):
    return math.dist((x1, y1), (x2, y2))

# calculate distances from starting points to compare later on
for i in range(0, width):
    for j in range(0, height):
        iter_x = (i * grid_side) + 1.5
        iter_y = (j * grid_side) + 1.5
        grid[j][i][0] = calculate_distance(iter_x, iter_y, starting_points[0][1] * grid_side, starting_points[0][0] * grid_side)
        grid[j][i][1] = calculate_distance(iter_x, iter_y, starting_points[1][1] * grid_side, starting_points[1][0] * grid_side)
        #grid[j][i][2] = calculate_distance(iter_x, iter_y, starting_points[2][1] * grid_side, starting_points[2][0] * grid_side)

In [239]:
def amount_equals_percentage(number_of_hits, p, variance):
    if number_of_hits == 0:
        return False
    return p - variance < (number_of_hits / number_of_simulations) < p + variance

# heuristically choose some radius r
r = (1 / a) * 100.0
p = 0.9
variance = 0.1

# check if a certain percentage of all hits are inside this radius, improve the radius until this is true
number_of_hits = 0
while True:
    for i in range(0, width):
        for j in range(0, height):
            for idx_sp in range(0, len(starting_points)):
                if grid[j][i][idx_sp] <= r:
                    number_of_hits += grid[j][i][idx_sp + 3]

    if amount_equals_percentage(number_of_hits, p, variance):
        break
    elif number_of_hits == 0 or (number_of_hits / number_of_simulations) < p:
        r = 1.5 * r
    elif (number_of_hits / number_of_simulations) > p:
        r = r / 2.0

    number_of_hits = 0

print("Found radius with percentage of " + str((number_of_hits / number_of_simulations) * 100) + "%: " + str(r))

Found radius with percentage of 87.78%: 33.788108825683594


In [240]:
# calculate map positions for radius


In [269]:
plt.rcParams['figure.dpi'] = 600
plt.rcParams['savefig.dpi'] = 600

circle1 = plt.Circle(starting_points[0], (r / grid_side), color='r')
circle2 = plt.Circle(starting_points[1], (r / grid_side), color='r')
#circle3 = plt.Circle(starting_points[2], (r[2] / grid_side), color='r')

#map = gis_incidences_nona.to_crs('epsg:3035')
map = gis_incidences_nona
map = map.get('geometry')

print(gpd.points_from_xy(x=[map[0].centroid.x], y=[map[0].centroid.y], crs="epsg:4326").to_crs('epsg:3035'))
print(gpd.points_from_xy(x=[map[132].centroid.x], y=[map[132].centroid.y], crs="epsg:4326").to_crs('epsg:3035'))
print(gpd.points_from_xy(x=[map[284].centroid.x], y=[map[284].centroid.y], crs="epsg:4326").to_crs('epsg:3035'))

sp1 = gpd.points_from_xy(x=[map[0].centroid.x], y=[map[0].centroid.y], crs="epsg:4326").to_crs('epsg:3035')
sp2 = gpd.points_from_xy(x=[map[132].centroid.x], y=[map[132].centroid.y], crs="epsg:4326").to_crs('epsg:3035')
sp3 = gpd.points_from_xy(x=[map[284].centroid.x], y=[map[284].centroid.y], crs="epsg:4326").to_crs('epsg:3035')

# we set one point anywhere in the grid and keep the relation between them for our simulation
print((sp1.x - sp2.x) / 1000)
print((sp1.y - sp2.y) / 1000)

# later on we can take the radius and project it onto the map of germany

#print(map)

# print(gpd.points_from_xy(x=starting_points[0][0], y=starting_points[0][1]))


#map = map.translate(xoff=-4492544, yoff=-3096577)

#map = map.scale(0.01, 0.01)
#print(map)



fig, ax = plt.subplots(1, 1,figsize=(20, 20))
ax.set_xlim((0, 1000))
ax.set_ylim((0, 1000))
ax.add_patch(circle1)
ax.add_patch(circle2)
#map.plot(ax=ax, aspect='equal')
#ax.add_patch(circle3)
ax.text(x=(starting_points[0][0] - 15), y=(starting_points[0][1] + 25), s=("p = " + str(p)), color="b", fontsize="xx-small")
ax.text(x=(starting_points[1][0] - 15), y=(starting_points[1][1] + 25), s=("p = " + str(p)), color="b", fontsize="xx-small")
#ax.text(x=(starting_points[2][0] - 15), y=(starting_points[2][1] + 10), s=("p = " + str(p)), color="w", fontsize="xx-small")
ax.plot(starting_points[0][0], starting_points[0][1], 'b+', markersize=12)
ax.plot(starting_points[1][0], starting_points[1][1], 'b+', markersize=12)
#ax.plot(starting_points[2][0], starting_points[2][1], 'b+', markersize=12)
plt.savefig("output.svg", format="svg", bbox_inches="tight")
#plt.show()

<GeometryArray>
[<POINT (4216100.631 2886984.404)>]
Length: 1, dtype: geometry
<GeometryArray>
[<POINT (4572975.017 2836348.325)>]
Length: 1, dtype: geometry
<GeometryArray>
[<POINT (4157937.834 3207368.394)>]
Length: 1, dtype: geometry
[-356.87438614]
[50.63607939]
